In [2]:
from scipy.io import wavfile
import numpy as np
import IPython.display as ipd
import plotly
import plotly.offline as pyo 
import plotly.graph_objs as go
import matplotlib.pyplot as plt 
import librosa
import soundfile as sf
import pandas as pd 
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from scipy.signal import argrelextrema

### Funciones

In [18]:
#Función para cortar los dataframes, el cambio de ritmo lo vi a ojo, debería hacer algo que detecte automáticamente. 
def corte(p0, pf, df):
  df=df[p0:pf]
  return df


#Función que me detecta los máximos, es decir, el ataque de cada golpe.
def maximos(df, columna, salto, fs, nombre_columna):
  max_indices = argrelextrema(df[columna].values, np.greater_equal, order=salto)[0]
  max_values = df[columna].iloc[max_indices]
  dfmax=pd.DataFrame(max_values)
  dfmax[nombre_columna] = dfmax.index / fs
  dfmax=dfmax.reset_index()
  return dfmax


#Función que me pone 0 en todas las amplitudes menos en los máximos donde pone 1.
def maximos2(df, columna, salto, fs, nombre_columna, columna_nueva):
    
    max_indices = argrelextrema(df[columna].values, np.greater_equal, order=salto)[0]

    df[columna + '_maximos'] = 0
 
    df.loc[df.index[max_indices], columna + '_maximos'] = 1

    dfmax = pd.DataFrame({
        columna + '_maximos': df[columna + '_maximos'],
        nombre_columna: df.index / fs
    })
    df[columna_nueva]=df[columna + '_maximos'].cumsum()

    return df

#Función para calcular la diferencia entre picos
def delta(df, columna):
    delta=pd.DataFrame(df[columna].diff())
    delta=delta.dropna()
    delta=delta.reset_index()
    return delta
#Guardo distancia entre picos metrónomo-tambor, cuando hay más del tambor guardo en "resto" el que no uso, cuando hay de menos
#guardo el que uso dos veces en "repetidos".
def procesar_picos(m, t):
    if len(t) > len(m):
        m['min_diferencia'] = None
        m['tiempo_tambor'] = None

        for i in range(len(m)):
            valor = m.loc[i, 'tiempo_metronomo']

            diferencias = t['tiempo_tambor'] - valor
            closest_index = (diferencias.abs()).idxmin()
            min_dif = diferencias.iloc[closest_index]

            m.loc[i, 'min_diferencia'] = min_dif
            m.loc[i, 'tiempo_tambor'] = t.loc[closest_index, 'tiempo_tambor']
            m.loc[i, 'amplitud_tambor'] = t.loc[closest_index, 'amplitud_tambor']

        resto = t[~t['tiempo_tambor'].isin(m['tiempo_tambor'])].copy()

        resto['distancia_al_mas_cercano'] = resto['tiempo_tambor'].apply(lambda x: abs(x - m['tiempo_metronomo']).min())

        tiempos = pd.concat([m['tiempo_metronomo'], m['tiempo_tambor'], m['min_diferencia'], m['amplitud_tambor']], axis=1)
        return tiempos, resto

    elif len(t) < len(m):
        m['min_diferencia'] = None
        m['tiempo_tambor'] = None
        t['seleccionado'] = 0

        for i in range(len(m)):
            valor = m.loc[i, 'tiempo_metronomo']

            diferencias = t['tiempo_tambor'] - valor
            closest_index = (diferencias.abs()).idxmin()
            min_dif = diferencias.iloc[closest_index]

            m.loc[i, 'min_diferencia'] = min_dif
            m.loc[i, 'tiempo_tambor'] = t.loc[closest_index, 'tiempo_tambor']
            m.loc[i, 'amplitud_tambor'] = t.loc[closest_index, 'amplitud_tambor']

            t.loc[closest_index, 'seleccionado'] += 1

        repetidos = t[t['seleccionado'] > 1].copy()

        repetidos['distancia_al_mas_cercano'] = repetidos['tiempo_tambor'].apply(lambda x: abs(x - m['tiempo_metronomo']).min())

        t.drop(columns=['seleccionado'], inplace=True)

        tiempos = pd.concat([m['tiempo_metronomo'], m['tiempo_tambor'], m['min_diferencia'], m['amplitud_tambor']], axis=1)
        return tiempos, repetidos

    else:
        m['min_diferencia'] = None
        m['tiempo_tambor'] = None

        for i in range(len(m)):
            valor = m.loc[i, 'tiempo_metronomo']

            diferencias = t['tiempo_tambor'] - valor
            closest_index = (diferencias.abs()).idxmin()
            min_dif = diferencias.iloc[closest_index]

            m.loc[i, 'min_diferencia'] = min_dif
            m.loc[i, 'tiempo_tambor'] = t.loc[closest_index, 'tiempo_tambor']
            m.loc[i, 'amplitud_tambor'] = t.loc[closest_index, 'amplitud_tambor']

        tiempos = pd.concat([m['tiempo_metronomo'], m['tiempo_tambor'], m['min_diferencia'], m['amplitud_tambor']], axis=1)
        return tiempos, None
#Función que agrega columna con suma acumulativa de otra.
def indice_sumatoria(df, columna, columna_nueva):
    df[columna_nueva]=df[columna].cumsum()
    return df
#Función donde calculo la desviación de un valor medio, no se porque le puse varianza
def varianza(df, columna, valor):
    df['varianza']=valor-df[columna]
    return df

def calcular_coef_variabilidad_local(df, columna, tamaño_ventana):
    resultados_df = pd.DataFrame()
    
    # Aplicar el cálculo por ventanas
    for i in range(len(df) - tamaño_ventana + 1):
        ventana = df[columna].iloc[i:i + tamaño_ventana]
        media_ventana = ventana.mean()
        desv_estandar_ventana = ventana.std()

        # Calcular el coeficiente de variabilidad local
        coef_var_local = desv_estandar_ventana / media_ventana if media_ventana != 0 else np.nan

        # Almacenar el resultado en el DataFrame
        resultados_df.loc[i + tamaño_ventana - 1, 'coef_var_local'] = coef_var_local

    return resultados_df


### Audios

In [4]:
#Señales de audio
audio_file = "sonido_1.wav"
audio_file2 = "sonido_2.wav"

#Disminuyo la frecuencia de muestreo para que no explote la compu
tambor, fst = librosa.load(audio_file, sr=8000)
metronomo, fsm = librosa.load(audio_file2, sr=8000)

### Código

In [5]:
#.describe() de mis datos 
tambor_ampl = pd.Series(tambor)
df_tambor_amplitud=pd.DataFrame(tambor_ampl.describe())
metronomo_ampl = pd.Series(metronomo)
df_metronomo_amplitud=pd.DataFrame(metronomo_ampl.describe())
df_datos=pd.concat([df_tambor_amplitud, df_metronomo_amplitud], axis=1)
df_datos.columns = ['ampl-tambor', 'ampl-metronomo']
df_datos

,ampl-tambor,ampl-metronomo
count,949975.000000,949975.000000
mean,-0.000054,-0.000036
std,0.034034,0.034758
min,-0.517822,-0.509436
25%,-0.006599,-0.006996
50%,-0.000038,-0.000022
75%,0.006481,0.006874
max,0.415741,0.515584


In [6]:
#Agrego una columna con el tiempo en segundos
dftambor=pd.DataFrame(tambor)
dftambor['tiempo_tambor'] = dftambor.index /fst
dftambor.columns = ['amplitud_tambor', 'tiempo_tambor']

dfmetronomo=pd.DataFrame(metronomo)
dfmetronomo['tiempo_metronomo'] = dfmetronomo.index /fsm
dfmetronomo.columns=['amplitud_metronomo', 'tiempo_metronomo']

In [7]:
#Corte de los dataframes en cuatro partes por los cambios de ritmo del metrónomo.
dftambor0=corte(32000, 195500, dftambor) 
dftambor1=corte(195000, 277500, dftambor)  
dftambor2=corte(277500, 443000, dftambor)
dftambor3=corte(443000, 535000, dftambor)
dftambor4=corte(537000, 860000, dftambor)
dftambor5=corte(861500, 934000, dftambor)

dfmetronomo0=corte(32000, 195000, dfmetronomo)
dfmetronomo1=corte(195000, 277500, dfmetronomo)
dfmetronomo2=corte(277500, 444000, dfmetronomo)
dfmetronomo3=corte(444000, 537000, dfmetronomo)
dfmetronomo4=corte(537000, 860000, dfmetronomo)
dfmetronomo5=corte(861500, 934000, dfmetronomo)

In [8]:
#Dataframe con los máximos
t_0=maximos(dftambor0, 'amplitud_tambor', 4500, fst, 'tiempo_tambor')
m_0=maximos(dfmetronomo0, 'amplitud_metronomo', 4500, fsm, 'tiempo_metronomo')
t_1=maximos(dftambor1, 'amplitud_tambor', 4500, fst, 'tiempo_tambor')
m_1=maximos(dfmetronomo1, 'amplitud_metronomo', 4500, fsm, 'tiempo_metronomo')
t_2=maximos(dftambor2, 'amplitud_tambor', 3250, fst, 'tiempo_tambor')
m_2=maximos(dfmetronomo2, 'amplitud_metronomo', 3250, fsm, 'tiempo_metronomo')
t_3=maximos(dftambor3, 'amplitud_tambor', 2000, fst, 'tiempo_tambor')
m_3=maximos(dfmetronomo3, 'amplitud_metronomo', 2000, fsm, 'tiempo_metronomo')
t_4=maximos(dftambor4, 'amplitud_tambor', 1500, fst, 'tiempo_tambor')
m_4=maximos(dfmetronomo4, 'amplitud_metronomo', 1500, fsm, 'tiempo_metronomo')
t_5=maximos(dftambor5, 'amplitud_tambor', 1250, fst, 'tiempo_tambor')
m_5=maximos(dfmetronomo5, 'amplitud_metronomo', 1250, fsm, 'tiempo_metronomo')

In [9]:
#Dataframes donde las amplitudes son 0 excepto en los máximos donde valen 1.
t_0g=maximos2(dftambor0, 'amplitud_tambor', 4500, fst, 'tiempo_tambor', 'indice')
m_0g=maximos2(dfmetronomo0, 'amplitud_metronomo', 4500, fsm, 'tiempo_metronomo', 'indice')
t_1g=maximos2(dftambor1, 'amplitud_tambor', 4500, fst, 'tiempo_tambor', 'indice')
m_1g=maximos2(dfmetronomo1, 'amplitud_metronomo', 4500, fsm, 'tiempo_metronomo', 'indice')
t_2g=maximos2(dftambor2, 'amplitud_tambor', 3250, fst, 'tiempo_tambor', 'indice')
m_2g=maximos2(dfmetronomo2, 'amplitud_metronomo', 3250, fsm, 'tiempo_metronomo', 'indice')
t_3g=maximos2(dftambor3, 'amplitud_tambor', 2000, fst, 'tiempo_tambor', 'indice')
m_3g=maximos2(dfmetronomo3, 'amplitud_metronomo', 2000, fsm, 'tiempo_metronomo', 'indice')
t_4g=maximos2(dftambor4, 'amplitud_tambor', 1500, fst, 'tiempo_tambor', 'indice')
m_4g=maximos2(dfmetronomo4, 'amplitud_metronomo', 1500, fsm, 'tiempo_metronomo', 'indice')
t_5g=maximos2(dftambor5, 'amplitud_tambor', 1250, fst, 'tiempo_tambor', 'indice')
m_5g=maximos2(dfmetronomo5, 'amplitud_metronomo', 1250, fsm, 'tiempo_metronomo', 'indice')

In [10]:
#Concateno los máximos de las cuatro partes del audio.
maximos_tambor=pd.concat([t_0,t_1, t_2, t_3, t_4, t_5], axis=0)
maximos_metronomo=pd.concat([m_0,m_1, m_2, m_3, m_4, m_5], axis=0)
maximos_tambor_norm=pd.concat([t_0g,t_1g, t_2g, t_3g, t_4g, t_5g], axis=0)
maximos_metronomo_norm=pd.concat([m_0g, m_1g, m_2g, m_3g, m_4g, m_5g], axis=0)


In [11]:
#Guardo tiempos por un lado, y restos y repetidos por el otro.
tiempos0, _ = procesar_picos(m_0, t_0)
tiempos0, resto0=procesar_picos(m_0, t_0)
tiempos1, _ = procesar_picos(m_1, t_1)
tiempos2, _ = procesar_picos(m_2, t_2)
tiempos2, repetidos2=procesar_picos(m_2, t_2)                           
tiempos3, _ = procesar_picos(m_3, t_3)
tiempos3, resto3=procesar_picos(m_3, t_3)
tiempos4, _ = procesar_picos(m_4, t_4)
tiempos4, resto4=procesar_picos(m_4, t_4)
tiempos5, _ = procesar_picos(m_5, t_5)
tiempos5, resto5=procesar_picos(m_5, t_5)
resto0['doble_golpe']=1
resto3['doble_golpe']=1
resto4['doble_golpe']=1
resto5['doble_golpe']=1
repetidos2['agujeros']=1


In [20]:
pd.options.display.max_rows = None
#Calculo la diferencia entre picos de cada parte
deltatambor0=delta(t_0, 'tiempo_tambor')
deltatambor0=varianza(deltatambor0, 'tiempo_tambor', deltametronomo0['tiempo_metronomo'].mean())
deltatambor1=delta(t_1, 'tiempo_tambor')
deltatambor1=varianza(deltatambor1, 'tiempo_tambor', deltametronomo1['tiempo_metronomo'].mean())
deltatambor2=delta(t_2, 'tiempo_tambor')
deltatambor2=varianza(deltatambor2, 'tiempo_tambor', deltametronomo2['tiempo_metronomo'].mean())
deltatambor3=delta(t_3, 'tiempo_tambor')
deltatambor3=varianza(deltatambor3, 'tiempo_tambor', deltametronomo3['tiempo_metronomo'].mean())
deltatambor4=delta(t_4, 'tiempo_tambor')
deltatambor4=varianza(deltatambor4, 'tiempo_tambor', deltametronomo4['tiempo_metronomo'].mean())
deltatambor5=delta(t_5, 'tiempo_tambor')
deltatambor5=varianza(deltatambor5, 'tiempo_tambor', deltametronomo5['tiempo_metronomo'].mean())

deltametronomo0=delta(m_0, 'tiempo_metronomo')
deltametronomo1=delta(m_1, 'tiempo_metronomo')
deltametronomo2=delta(m_2, 'tiempo_metronomo')
deltametronomo3=delta(m_3, 'tiempo_metronomo')
deltametronomo4=delta(m_4, 'tiempo_metronomo')
deltametronomo5=delta(m_5, 'tiempo_metronomo')


## Guardado de dataframes

In [24]:
#Guardo los dataframes.
dftambor.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dftambor.csv', index=False)
dfmetronomo.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dfmetronomo.csv', index=False)

dftambor0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dftambor0.csv', index=False)
dfmetronomo0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dfmetronomo0.csv', index=False)
dftambor1.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dftambor1.csv', index=False)
dfmetronomo1.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dfmetronomo1.csv', index=False)
dftambor2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dftambor2.csv', index=False)
dfmetronomo2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dfmetronomo2.csv', index=False)
dftambor3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dftambor3.csv', index=False)
dfmetronomo3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dfmetronomo3.csv', index=False)
dftambor4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dftambor4.csv', index=False)
dfmetronomo4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dfmetronomo4.csv', index=False)
dftambor5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dftambor5.csv', index=False)
dfmetronomo5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/dfmetronomo5.csv', index=False)
                   
tiempos0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/tiempos0.csv', index=False)
tiempos1.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/tiempos1.csv', index=False)
tiempos2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/tiempos2.csv', index=False)
tiempos3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/tiempos3.csv', index=False)
tiempos4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/tiempos4.csv', index=False)
tiempos5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/tiempos5.csv', index=False)

resto0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/resto0.csv', index=False)
repetidos2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/repetidos2.csv', index=False)
resto3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/resto3.csv', index=False)
resto4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/resto4.csv', index=False)
resto5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/resto5.csv', index=False)

t_0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_0.csv', index=False)
t_1.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_1.csv', index=False)
t_2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_2.csv', index=False)
t_3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_3.csv', index=False)
t_4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_4.csv', index=False)
t_5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_5.csv', index=False)
m_0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_0.csv', index=False)
m_1.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_1.csv', index=False)
m_2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_2.csv', index=False)
m_3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_3.csv', index=False)
m_4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_4.csv', index=False)
m_5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_5.csv', index=False)

t_0g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_0g.csv', index=False)
t_1g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_1g.csv', index=False)
t_2g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_2g.csv', index=False)
t_3g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_3g.csv', index=False)
t_4g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_4g.csv', index=False)
t_5g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/t_5g.csv', index=False)
m_0g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_0g.csv', index=False)
m_1g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_1g.csv', index=False)
m_2g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_2g.csv', index=False)
m_3g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_3g.csv', index=False)
m_4g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_4g.csv', index=False)
m_5g.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/m_5g.csv', index=False)
           
maximos_tambor.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/maximos_tambor.csv', index=False)
maximos_metronomo.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/maximos_metronomo.csv', index=False)
maximos_tambor_norm.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/maximos_tambor_norm.csv', index=False)
maximos_metronomo_norm.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/maximos_metronomo_norm.csv', index=False)

deltatambor0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltatambor0.csv', index=False)
deltatambor1.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltatambor1.csv', index=False)
deltatambor2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltatambor2.csv', index=False)
deltatambor3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltatambor3.csv', index=False)
deltatambor4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltatambor4.csv', index=False)
deltatambor5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltatambor5.csv', index=False)
deltametronomo0.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltametronomo0.csv', index=False)
deltametronomo1.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltametronomo1.csv', index=False)
deltametronomo2.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltametronomo2.csv', index=False)
deltametronomo3.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltametronomo3.csv', index=False)
deltametronomo4.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltametronomo4.csv', index=False)
deltametronomo5.to_csv('H:/Mi unidad/Análisis de audio/Audio 1/deltametronomo5.csv', index=False)

,index,tiempo_tambor
0,1,0.906250
1,2,0.932625
2,3,1.023500
3,4,0.959875
4,5,0.983750
5,6,0.997125
6,7,1.010500
7,8,1.037875
8,9,0.955375
9,10,0.959500
